# Vergleich: Numerische Integration
Dieses Notebook ist eine Beilage zur Facharbeit: *Vergleich numerischer Verfahren zur Modellierung des Dreikörperproblems*  
---
In diesem Notebook sollen verschiedene numerische Methoden zum Integrieren einer Differentialgleichung eingesetzt werden: 
1. $y' = f(x,y) = cos(x)$  
Lösung:
$y=sin(x)$.  
2. $y' = f(x,y) = e^{cos(x)} \cdot (2x \cdot cos(x) - sin(x) \cdot sin(x^2))$  
Lösung:
$y=e^{cos(x)} \cdot sin(x^2)$.  
---
Folgende Verfahren sollen dabei zum Einsatz kommen und in ihrer Genauigkeit verglichen werden:

*   Euler-Verfahren
*   Euler-Cromer-Verfahren
*   Heun-Verfahren
*   Runge-Kutta 2er Ordnung
*   Runge-Kutta 4er Ordnung
*   Bulirsch-Stoer-Verfahren
---
Im Folgenden wird die Schrittweite $h$, die Funktion $y(x)$, die Ableitung $f(x,y)$ und die Iteration mit $i$ dargestellt. Eine Erklärung zu den Verfahren befindet sich in der Facharbeit. Alle Verfahren, bis auf das Bulirsch-Stoer-Verfahren, wurden vollständig ohne Hilfe implementiert. Beim Bulirsch-Stoer-Verfahren, diente das Buch "*Numerical Methods in Engineering with Python 3*" (*Jaan Kiusalaas*) als Stütze.

In [ ]:
from mpmath import * # Nutzung von Hoch-Präzisions-Arithmetik (siehe https://mpmath.org/)

## Euler-Verfahren
$$y_{i+1} = y_i+ hf(x_i,y_i)$$

In [ ]:
# f: DGL
# startPoint: (x,y) Anfangswertepaar
# stepSize: Schrittegröße
# steps: Anzahl an Schritten
def eulerMethod(f,startPoint,stepSize,steps):
  # Liste an Punkten: [(x0,y0),...,(xn,yn)]
  result = [startPoint,]
  # Aktuelle Approximierung
  pi = startPoint
  # Iteration über alle Schritte
  for i in range(steps):
    # Hinzufügen einer neuen Approximierung (xi+1,yi+1), gemäß dem Verfahren
    result.append(
        (
         pi[0]+stepSize,               # x-Koordinate
         pi[1]+stepSize*f(pi[0],pi[1]) # y-Koordinate
        )
    )
    # Aktualisierung der akutellen Approximierung
    pi = result[i+1]
  # Rückgabe der Approximierungen und einer Identifikation
  return result,"Euler"

## Euler-Cromer-Verfahren
$$y_{i+1} = y_i+ hf(x_i+h,y_i + hf(x_i,y_i))$$

In [ ]:
# f: DGL
# startPoint: (x,y) Anfangswertepaar
# stepSize: Schrittegröße
# steps: Anzahl an Schritten
def eulerCromerMethod(f,startPoint,stepSize,steps):
  # Liste an Punkten: [(x0,y0),...,(xn,yn)]
  result = [startPoint,]
  # Aktuelle Approximierung
  pi = startPoint
  # Iteration über alle Schritte
  for i in range(steps):
    # Hinzufügen einer neuen Approximierung (xi+1,yi+1), gemäß dem Verfahren
    result.append(
        (
         pi[0]+stepSize,                                      # x-Koordinate
         pi[1]+stepSize*f(pi[0]+stepSize,pi[1]+f(pi[0],pi[1]))# y-Koordinate
        )
    )
    # Aktualisierung der akutellen Approximierung
    pi = result[i+1]
  # Rückgabe der Approximierungen und einer Identifikation
  return result,"Euler-Cromer"

## Heun-Verfahren
$$y_{i+1} = y_i+ \frac{h}{2}(f(x_i,y_i)+f(x_i+h,y_i+hf(x_i,y_i)))$$

In [ ]:
# f: DGL
# startPoint: (x,y) Anfangswertepaar
# stepSize: Schrittegröße
# steps: Anzahl an Schritten
def heunMethod(f,startPoint,stepSize,steps):
  # Liste an Punkten: [(x0,y0),...,(xn,yn)]
  result = [startPoint,]
  # Aktuelle Approximierung
  pi = startPoint
  # Iteration über alle Schritte
  for i in range(steps):
    # Hinzufügen einer neuen Approximierung (xi+1,yi+1), gemäß dem Verfahren
    result.append(
        (
         pi[0]+stepSize,
         pi[1]+stepSize/2*(f(pi[0],pi[1])+f(pi[0]+stepSize,pi[1]+stepSize*f(pi[0],pi[1])))
        )
    )
    # Aktualisierung der akutellen Approximierung
    pi = result[i+1]
  # Rückgabe der Approximierungen und einer Identifikation
  return result,"Heun"

## Runge-Kutta 2er Ordnung 
Dieses Verfahren ist etwas präziser als das Heun-Verfahren, da die "echte" Steigung auf der Hälfte zum nächsten Zeitschritt errechnet wird und nicht der Mittelwert zweier Steigungen genutzt wird. Dieser "echte" Wert, wird mithilfe des Euler-Verfahrens angenährt, indem ein halber Schritt gegangen wird.
$$K_0 = hf(x,y)$$
$$K_1 = hf(x+\frac{h}{2},y+\frac{K_0}{2})$$
$$y_{i+1} = y_i + K_1$$


In [ ]:
# f: DGL
# startPoint: (x,y) Anfangswertepaar
# stepSize: Schrittegröße
# steps: Anzahl an Schritten
def rk2(f,startPoint,stepSize,steps):
  # Liste an Punkten: [(x0,y0),...,(xn,yn)]
  result = [startPoint]
  # Aktuelle Approximierung
  pi = startPoint
  # Iteration über alle Schritte
  for i in range(steps):
    # Runge-Kutta-Terme
    k0 = stepSize * f(pi[0],pi[1])
    k1 = stepSize * f(pi[0]+stepSize/2,pi[1]+k0/2)
    # Hinzufügen einer neuen Approximierung (xi+1,yi+1), gemäß dem Verfahren
    result.append((pi[0]+stepSize,pi[1]+k1))
    # Aktualisierung der akutellen Approximierung
    pi = result[i+1]
  # Rückgabe der Approximierungen und einer Identifikation
  return result,"RK2"

## Runge-Kutta 4er Ordnung 
$$K_0 = hf(x,y)$$
$$K_1 = hf(x+\frac{h}{2},y+\frac{K_0}{2})$$
$$K_2 = hf(x+\frac{h}{2},y+\frac{K_1}{2})$$
$$K_3 = hf(x+h,y+K_2)$$
$$y_{i+1} = y_i + \frac{1}{6}(K_0 + 2K_1 + 2K_2 + K_3)$$

In [ ]:
# f: DGL
# startPoint: (x,y) Anfangswertepaar
# stepSize: Schrittegröße
# steps: Anzahl an Schritten
def rk4(f,startPoint,stepSize,steps):
  # Liste an Approximierungen
  result = [startPoint]
  # Aktuelle Approximierung
  pi = startPoint
  # Iteration über alle Schritte
  for i in range(steps):
    # Runge-Kutta-Terme
    k0 = stepSize * f(pi[0],pi[1])
    k1 = stepSize * f(pi[0]+stepSize/2,pi[1]+k0/2)
    k2 = stepSize * f(pi[0]+stepSize/2,pi[1]+k1/2)
    k3 = stepSize * f(pi[0]+stepSize, pi[1]+k2)
    # Hinzufügen einer neuen Approximierung (xi+1,yi+1), gemäß dem Verfahren
    result.append((pi[0]+stepSize,pi[1]+1/6*(k0+2*k1+2*k2+k3)))
    # Aktualisierung der akutellen Approximierung
    pi = result[i+1]
  # Rückgabe der Approximierungen und einer Identifikation
  return result,"RK4"

## Bulirsch-Stoer 
(siehe "*Numerical Methods in Engineering with Python 3*" (*Jaan Kiusalaas*))  
Auf Basis der *Richardson-Extrapolation* und dem *Modifizierten-Mittelpunkt-Verfahren*.
### Modifiziertes Mittelpunkt Verfahren
Erster Schritt:  
$$y_1 = y_0 + h f(x_0,y_0)$$
Alle anderen Schritt:  
$$y_{i+1} = y_{i-1} + 2hf(t_i,y_i)$$  
Letzter Schritt:  
$$y_{n-1} = \frac{1}{2}\cdot (y_{n-2} + y_{n-3} + hf(t_{n-2},y_{n-2}))$$

In [ ]:
# NACH: *Numerical Methods in Engineering with Python 3*
# f: DGL
# startPoint: (x,y) Anfangswertepaar
# xStop: Endwert für x
# tol: Toleranz des maximalen Fehlers
def midpointExtrapolation(f,startPoint,xStop,tol):
  # Funktion für den "normalen" Integrationsprozess
  # (modifiziertes Mittelpunkt Verfahren)
  def modifiedMidpoint(f,startPoint,xStop,steps):
    # Ermitteln der Schrittweite
    stepSize = (xStop - startPoint[0])/steps
    # Festlegen des x-Wertes
    x = startPoint[0]
    # Festlegen des ersten y-Wertes
    y0 = startPoint[1]
    # Berechnen des zweiten y-Wertes (mit Euler-Verfahren)
    y1 = y0 + stepSize*f(startPoint[0],y0)
    # Iteration über die restlichen Schritte
    for i in range(steps-1):
      # Aktualisieren des x-Wertes
      x=x+stepSize
      # Berechnen des aktuellen y-Wertes
      y2 = y0 + 2.0*stepSize*f(x,y1)
      # Aktualisieren des vorletzten y-Wertes
      y0 = y1
      # Aktualisieren des letzten y-Wertes
      y1 = y2
    # Zurückgeben des letzten y-Wertes gemäß des modifizierten Mittelpunkt Verfahrens
    return 0.5*(y1 + y0 + stepSize*f(x,y2))
  # Richardson-Extrapolation der berechneten Werte
  def richardson(r,k):
    # Iteration über alle bisherigen Ergbnisse
    for j in range(k-1,0,-1):
      # Berechnen des konstanten Faktor (siehe Richardson-Extrapolations-Formel)
      const = (k/(k - 1.0))**(2.0*(k-j))
      # Zuweisen eines neuen Wertes (mutieren der Liste)
      r[j] = (const*r[j+1] - r[j])/(const - 1.0)
  # Maximaler Wert für k (2*k := Unterteilungen des Intervalls [x,x+h])
  kMax = 51
  # Liste von 51 Werten (=> Liste der maximal 51, auf 0 extrapolierten Werte)
  r = np.zeros(kMax)
  # Start mit 2 Sub-Divisionen des Integrationsintervalls 
  n = 2
  # Erste Integration mit 2 Sub-Divisionen
  r[1] = modifiedMidpoint(f,startPoint,xStop,n)
  # Speichern des letzten genausten r's
  lastR = r[1].copy()
  # Erhöhen der Sub-Divisionen (n) um 2
  # => Wiederholtes Extrapolieren auf Schrittgröße 0 (n->inf)
  # Wenn die Differenz des vorherigen extrapolierten Wertes und des aktuellen Wertes,
  # kleiner als tol ist, dann ist der Integrationsschritt abgeschlossen
  for k in range(2,kMax):
    # Festlegen der Anzahl an Sub-Divisionen
    n = 2*k
    # Integration mit n Schritten
    r[k] = modifiedMidpoint(f,startPoint,xStop,n)
    # Extrapolieren mit der Richardson-Extrapolation
    richardson(r,k)
    # Berechnen des Fehlers
    e = sqrt(np.sum((r[1] - lastR)**2))
    # Überprüfen der Toleranz-Schwelle
    if e < tol:
      # Wenn Toleranz unterschritten wurde, wird der extrapolierte Wert zurückgegeben
      return r[1]
    # Festelgen des letzten genausten r's
    lastR = r[1].copy()

In [ ]:
# f: DGL
# startPoint: (x,y) Anfangswertepaar
# xStop: Endwert für x
# stepSize: Schrittegröße
# tol: Toleranz des maximalen Fehlers
def bulirschStoerMethod(f,startPoint,xStop,stepSize,tol=1.0e-6):
  # Liste an Approximierungen
  result = [startPoint]
  # Aktuelle Approximierung
  x = startPoint[0]
  y = startPoint[1]
  # Iteration bis xStop erreicht wird
  while x < xStop:
    # Integrieren & Extrapolieren, für einen neuen y-Wert unter Toleranz
    y = midpointExtrapolation(f,(x,y),x+stepSize,tol)
    # Berechnen der neuen x-Koordinate
    x = x + stepSize
    result.append((x,y))
  # Rückgabe der Approximierungen und einer Identifikation
  return result, "Bulirsch-Stoer mit Toleranz: {:e}".format(tol)

## Integrationsprozess


In [ ]:
# Importieren von Bibliotheken
# Numpy für das Benutzen von speziellen Arrays
import numpy as np 
# Matplotlib für die Darstellung von Daten in einem Plot
import matplotlib.pyplot as plt
# Anzeigen eines Plots, direkt im Jupyter Notebook ("inline")
%matplotlib inline

In [ ]:
# Einstellen der Anzahl an Nachkommastellen der Gleitkommazahlen
mp.dps = 50
# Differentialgleichung I
# f = lambda x,y: cos(x)
# Differentialgleichung II
f = lambda x,y: e**(mp.cos(x)) * (2*x*mp.cos(x*x) - sin(x) * sin(x*x))
# Lösung von DGL I
# solution = lambda x: sin(x)
# Lösung von DGL II
solution = lambda x: e**(mp.cos(x)) * mp.sin(x*x)
# Anfangswertepaar
startPoint = np.array([mpf(0),mpf(0)])
startPoint[1] = solution(startPoint[0])
# Schrittgröße
stepSize = 1e-3
# Anzahl an Schritten
steps = 10000
# Erstellung von einer Lösung (x-Wert-Liste, y-Wert-Liste)
x = np.linspace(0,stepSize*steps+1,100)
y = [solution(px) for px in x]

In [ ]:
# Liste an Integrations-Verfahren, welche eingesetzt werden
integrators = [eulerMethod,eulerCromerMethod,heunMethod,rk2,rk4]

In [ ]:
# Liste an Ergebnissen
results = []
# Ausführen der Integration für alle Integrations-Verfahren
for integrator in integrators:
  result = integrator(f,startPoint,stepSize,steps)
  results.append(result)

In [ ]:
# Integration mit dem Bulirsch-Stoer-Verfahren (tol = 1e-10)
results.append(bulirschStoerMethod(f,startPoint,steps*stepSize,stepSize,tol=1e-10))

In [ ]:
# Integration mit dem Bulirsch-Stoer-Verfahren (tol = 1e-6)
results.append(bulirschStoerMethod(f,startPoint,steps*stepSize,stepSize,tol=1e-6))

## Vergleich der Ergebnisse

In [ ]:
# 10x10 Plotsize
plt.rcParams['figure.figsize'] = [10, 10]
# Erstellen einer Figur
fig = plt.figure()
# Hinzufügen eines Plots
ax = fig.add_subplot()
# Liste von verschiedenen Linientypen (für bessere Visibilität von überlappenden Linien)
lines = [(0, (1, 0.5)),"--"]
# Iteration über alle Ergebnisse
for result in range(len(results)):
  # Plotten jedes Ergebnisses
  ax.plot([p[0] for p in results[result][0]],[abs(p[1]-solution(p[0])) for p in results[result][0]],label=results[result][1],ls=lines[result % len(lines)],lw=2.7)
# Logarithmische Skalierung
ax.set_yscale('log')
# Beschriften 
#ax.set_ylabel('$|\cos{x}- y(x)_{num}|$',fontsize=16)
ax.set_ylabel('$|e^{\cos{x}} \cdot \sin{x^2}- y(x)_{num}|$',fontsize=16)
ax.set_xlabel('x-Achse',fontsize=16)
ax.set_title('Numerisches Lösen mit verschiedenen Integrationsverfahren',fontsize=16, pad=20)
ax.legend()
plt.show()
# Speichern der Figur
fig.savefig('dgl-abs-error.pdf')

Ergebnisse:  
<img src="https://drive.google.com/uc?export=view&id=1sti3BslSekD1Sy3YZL4myjEPnp-s2Dr1" width="400"/>
<img src="https://drive.google.com/uc?export=view&id=1Tzgjv0Vv_FGfwnDA6mJamcSsMg_eQbk1" width="400"/>

## Abhängigkeit des Fehlers von der Schrittgröße
Hierbei soll die Abhängigkeit bzw. das Wachstum des Fehlers bei einer Veränderung der Schrittgröße untersucht werden. 

In [ ]:
import math
# Liste an zu untersuchenden Schrittgrößen
stepSizes = [1e-4,2e-4,3e-4,4e-4,5e-4]
# Letzter x-Wert
xStop = 10
# Ergebnisse mit den verschiedenen Schrittgrößen
stepSizeResults = []
# Liste an Integraten, welche genutzt werden sollen
integrators = [eulerCromerMethod,heunMethod,rk2,rk4]
# Iteration über alle Integratoren
for integrator in integrators:
  # Liste an Ergebnissen aller Integratoren
  res = []
  # Iteration über alle Schrittgrößen
  for stepSize in stepSizes:
    # Integrieren mit jeweiligem Integrator + Schrittgröße
    r = integrator(f,startPoint,stepSize,math.ceil(xStop/stepSize))
    # Hinzufügen zu Liste an Ergebnissen dieses Integrators
    res.append(r[0])
  # Hinzufügen der Ergebnisse dieses Integrators, zu allgemeinen Ergebnissen
  stepSizeResults.append([res,r[1]])

Der Mittelwert des absoluten Fehlers soll berechnet werden:

In [ ]:
# Liste der Mittelwerte
avgErr = []
# Iteration über alle Integrationen
for i in range(len(stepSizeResults)):
  # Liste der Mittelwerte für das iterierte Verfahren wird hinzugefügt
  avgErr.append([])
  for j in range(len(stepSizeResults[i][0])):
    avgErr[i].append(sum([abs(p[1]-solution(p[0])) for p in stepSizeResults[i][0][j]])/len(stepSizeResults[i][0][j]))

In [ ]:
# Ändern der Größe des Plots
plt.rcParams['figure.figsize'] = [10, 10]
# Erstellen eines Plots
fig,ax = plt.subplots(2,2)
for result in range(len(stepSizeResults)):
  ax[result//2][result%2].plot(stepSizes,avgErr[result],label=stepSizeResults[result][1], alpha=1 ,lw=2.7)
  ax[result//2][result%2].set_title(stepSizeResults[result][1])
  ax[result//2][result%2].set_ylabel('$\overline{err}_{abs}$')
  ax[result//2][result%2].set_xlabel('Schrittgröße $h$')
  ax[result//2][result%2].ticklabel_format(axis="x", style="sci", scilimits=(0,0))
fig.tight_layout()
plt.show()
fig.savefig('stepSizes.pdf')

Ergebnisse:  
<img src="https://drive.google.com/uc?export=view&id=1Fq8SVkBC1NUUqqIOnfvo5s0EnNcSVsEA" width="400"/>